# ***Installation Requirements***



In [ ]:
!pip install tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 32.1 MB/s eta 0:00:00
 

# ***Mount Google Drive***

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/ieee_tnsm25_tes-models/TES-RNN/Capacity_Forecasting

Mounted at /gdrive
/gdrive/My Drive/ieee_tnsm25_tes-models/TES-RNN/Capacity_Forecasting


# ***Imports***

In [ ]:
import os
import math
import time
import torch
import pickle
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from data_loading import create_dataset, Dataset
from config import get_config
from trainer import TESRNNTrainer
from validator import TESRNNValidator
from tester import TESRNNTester
from model import TESRNN
from loss_modules import *

# ***TES-RNN***

In [ ]:
# CONFIGURATION SETTINGS

# List of the services to be tested
services = ['Facebook', 'Instagram', 'Snapchat']

# Number of clusters types
num_clusterss = [1]

# List of alphas to be tested
alphas = [1, 2, 3, 5]

# Define the number of training epochs
epochs = 20

# Define the number of training batch size
batch_size = 288

# Define the number of train, validation and test samples
train_samples = 16128
val_samples = 4032
test_samples = 2016

# Define the input size and output size of the prediction
input_size = 6
output_size = 1

# Define simulation run seed
num_run = 0
torch.manual_seed(num_run)

# Define the device type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ACTOR-CRITIC IMPORTS AND SETTINGS
from sac import SAC
from Utils.buffer import ReplayBuffer
from torch.autograd import Variable

# Define state, action size and agents
state_size = 2
action_size = 21
possible_tau = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
num_agents = 1

# Define general parameters
gamma = 0.99
tau_ac = 0.001
rollout_threads = 1
ac_batch_size = 10
pol_hidden_dim = 512
critic_hidden_dim = 512
pi_lr = 0.01
q_lr = 0.01
norm_rews = True

# Define Actor-Critic iterations and experience-buffer length
ac_iterations = 50
buffer_length = ac_iterations

In [ ]:
# SIMULATION RUNS

# Iterate over the services
for service in services:

    # Iterate over the number of clusters types
    for num_clusters in num_clusterss:

        # Iterate over the alphas
        for alpha in alphas:

            # Iterate for the number of clusters
            for run in range(num_clusters):

                # Obtain the data of the simulation
                config = get_config('Traffic', epochs, num_clusters, batch_size, train_samples, val_samples, test_samples, alpha, input_size, output_size)
                if num_clusters > 1:
                    data = '../../../Dataset/' + service + '/time_load_cor_matrix_' + str(num_clusters) + '_clust.npy'
                    train, val, test = create_dataset(data, config['chop_train'], config['chop_val'], config['chop_test'], clustering=True, cluster=run)
                else:
                    data = '../../../Dataset/' + service + '/time_load_cor_matrix.npy'
                    train, val, test = create_dataset(data, config['chop_train'], config['chop_val'], config['chop_test'])


                # Get the maximums and the datasets
                max_train = np.max(train[0])
                max_val = np.max(val[0])
                max_test = np.max(test[0])
                dataset = Dataset(train, val, test, config['device'])

                # Set the maximum as the maximum in the training set
                maximum = max_train


                # Get the average of the validation traffic val
                avg_val = np.mean(val[0])
                std_val = np.std(val[0])

                # Single model per configuration case
                max_avg_val = avg_val
                max_std_val = std_val

                # Turn on exploration
                exploration = True

                # Actor Critic initialization
                ac_model = SAC.init_from_env(state_size, action_size, num_agents, gamma, tau_ac, pi_lr=pi_lr, q_lr=q_lr, pol_hidden_dim=pol_hidden_dim, critic_hidden_dim=critic_hidden_dim)
                replay_buffer = ReplayBuffer(buffer_length, num_agents, [(state_size) for j in range(num_agents)], [action_size for k in range(num_agents)])

                # Prepare the model
                ac_model.prep_rollouts(device='cpu')

                for ac_iter in range(ac_iterations):

                    # Check whether turn off exploration
                    if ac_iter >= (ac_iterations - 5) :
                        exploration = False

                    # Get the average value in the validation dataset (normalized state of the Actor Critic)
                    avg_validation_traffic = avg_val / max_avg_val

                    # Get the standard deviation value in the validation dataset (normalized state of the Actor Critic)
                    std_validation_traffic = std_val / max_std_val


                    # Determine the state, action and next state for the Actor Critic
                    complete_agent_state = np.ndarray(shape=(rollout_threads, num_agents), dtype=object)
                    agent_state = np.ndarray(shape=(state_size,))
                    agent_state[0] = avg_validation_traffic
                    agent_state[1] = std_validation_traffic
                    for k in range(num_agents):
                        complete_agent_state[0,k] = agent_state
                    torch_state = [Variable(torch.Tensor(np.vstack(complete_agent_state[:, j])), requires_grad=False) for j in range(num_agents)]


                    # Get the action from the Actor Critic
                    torch_action = ac_model.step(torch_state, explore=exploration)
                    agent_actions = [ac.data.numpy() for ac in torch_action]
                    tau = possible_tau[np.argmax(agent_actions[0])]

                    # Dataloader initialization
                    dataloader = DataLoader(dataset, batch_size=config['series_batch'], shuffle=False)

                    # Model initialization
                    run_id = service + '/Alpha_' + str(alpha) + '/Simulation_' + str(num_run)
                    model = TESRNN(tau = tau, maximum = maximum, num_clusters = num_clusters, config = config, run_id = run_id)

                    # Run model trainer
                    trainer = TESRNNTrainer(model, dataloader, run_id, config)
                    trainer.train_epochs()

                    # Run model validator
                    validator = TESRNNValidator(model, dataloader, run_id, config)
                    validator.validating()

                    # Compute denormalized validation loss
                    norm_preds = np.load('Results/' + run_id + '/val_predictions.npy')
                    norm_actuals = np.load('Results/' + run_id + '/val_actuals.npy')
                    levels = np.load('Results/' + run_id + '/val_levels.npy')
                    val_loss = denorm_validation_loss(norm_preds, norm_actuals, levels, alpha)



                    # Determine the next state for the Actor Critic
                    complete_agent_post_state = np.ndarray(shape=(rollout_threads, num_agents), dtype=object)
                    agent_post_state = np.ndarray(shape=(state_size,))
                    agent_post_state[0] = avg_validation_traffic
                    agent_post_state[1] = std_validation_traffic
                    for k in range(num_agents):
                        complete_agent_post_state[0,k] = agent_post_state
                    torch_post_state = [Variable(torch.Tensor(np.vstack(complete_agent_post_state[:, j])), requires_grad=False) for j in range(num_agents)]
                    rewards = np.ndarray(shape=(rollout_threads, num_agents))
                    rewards[0,0] = -val_loss


                    # Save the experience in the replay buffer
                    replay_buffer.push(complete_agent_state, agent_actions, rewards, complete_agent_post_state)


		            # Perform training of the Radio agent model
                    if len(replay_buffer) >= (ac_batch_size) and ac_iter < (ac_iterations - 5):
                        ac_model.prep_training(device='cpu')
                        sample = replay_buffer.sample(ac_batch_size, to_gpu=False, norm_rews=norm_rews)
                        ac_model.update_critic(sample)
                        ac_model.update_policies(sample)
                        ac_model.update_all_targets()
                        ac_model.prep_rollouts(device='cpu')

                        # Perform a sample inference
                        complete_agent_state = np.ndarray(shape=(rollout_threads, num_agents), dtype=object)
                        agent_state = np.ndarray(shape=(state_size,))
                        agent_state[0] = avg_validation_traffic
                        agent_state[1] = std_validation_traffic
                        for k in range(num_agents):
                            complete_agent_state[0,k] = agent_state
                        torch_state = [Variable(torch.Tensor(np.vstack(complete_agent_state[:, j])), requires_grad=False) for j in range(num_agents)]
                        torch_action = ac_model.step(torch_state, explore=False)
                        agent_actions = [ac.data.numpy() for ac in torch_action]
                        test_tau = possible_tau[np.argmax(agent_actions[0])]
                        np.save('Results/' + run_id + '/test_tau_ac_iter_' + str(ac_iter) + '.npy', tau)


                # Save the model
                file_path = os.path.join('AC_Models/', run_id)
                model_path = os.path.join(file_path, 'ac_model_sim_' + str(num_run))
                os.makedirs(file_path, exist_ok=True)
                ac_model.save('AC_Models/' + run_id + '/ac_model_sim_' + str(num_run))



                # Run the optimized model after taus optimization

                # Dataloader initialization
                dataloader = DataLoader(dataset, batch_size=config['series_batch'], shuffle=False)

                # Model initialization
                run_id = service + '/Alpha_' + str(alpha) + '/Simulation_' + str(num_run)
                model = TESRNN(tau = tau, maximum = maximum, num_clusters = num_clusters, config = config, run_id = run_id)

                # Run model trainer
                trainer = TESRNNTrainer(model, dataloader, run_id, config)
                trainer.train_epochs()

                # Run model tester
                tester = TESRNNTester(model, dataloader, run_id, config, service, num_clusters, run)
                predictions, actuals = tester.testing()

                # Move to numpy arrays
                predictions = predictions.cpu()
                actuals = actuals.cpu()

                # Denormalize the predictions and actuals
                levels = np.load('Results/' + run_id + '/test_levels.npy')
                predictions = predictions[:,0,0] * levels
                actuals = actuals[:,0,0] * levels

                # Find the peak
                peak = torch.max(actuals)

                # Move to numpy arrays
                predictions = predictions.cpu().numpy()
                actuals = actuals.cpu().numpy()
                peak = peak.cpu().numpy()

                # Find the different parts of alphaloss
                den_loss, over, sla = evaluate_costs_single_clust(predictions, actuals, peak, alpha)
                sla_cost = den_loss - over
                print("Denormalized Alpha-loss: ", den_loss)


                # Store the results
                np.save('Results/' + run_id + '/tes-rnn_predictions_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), predictions)
                np.save('Results/' + run_id + '/tes-rnn_actuals_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), actuals)
                np.save('Results/' + run_id + '/tes-rnn_alpha_loss_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), den_loss)
                np.save('Results/' + run_id + '/tes-rnn_over_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), over)
                np.save('Results/' + run_id + '/tes-rnn_sla_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), sla_cost)
                np.save('Results/' + run_id + '/tes-rnn_tau_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), tau)


Denormalized Alpha-loss:  6273716225023.168


KeyboardInterrupt: 